In [1]:
import numpy as np
import cv2

In [2]:
img_trans_a = cv2.imread('transA.jpg')
img_trans_b = cv2.imread('transB.jpg')
img_sim_a = cv2.imread('simA.jpg')
img_sim_b = cv2.imread('simB.jpg')
img_trans_a_gray = cv2.cvtColor(img_trans_a, cv2.COLOR_BGR2GRAY)
img_trans_b_gray = cv2.cvtColor(img_trans_b, cv2.COLOR_BGR2GRAY)
img_sim_a_gray = cv2.cvtColor(img_sim_a, cv2.COLOR_BGR2GRAY)
img_sim_b_gray = cv2.cvtColor(img_sim_b, cv2.COLOR_BGR2GRAY)

In [3]:
R_THRESHOLD = 150
HARRIS_WIN_SIZE = 11
R_RADIUS = 11
HIST_BINS = 8

1 Harris corners

In class and in the text we have developed the Harris operator. To find the Harris points you need to compute the gradients in both the X and Y directions. These will probably have to be lightly filtered using a Gaussian to be well behaved. You can do this either the “naive” way - filter the image and then do simple difference between left and right (X gradient) or up and down (Y gradient) - or you can take an analytic derivative of a Gaussian in X or Y and use that filtter. The scale of the filtering is up to you. You may play with the size of the Gaussian as it will interact window size of the corner detection.

1.1 Write functions to compute both the X and Y gradients. Try your code on both transA and simA. To display the output, adjoin the two gradient images(X and Y) to make a new, twice as wide, single image (the ”gradient-pair”). Since gradients have negaitve and positiive values, you’ll need to produce and image that is gray for 0.0 and balck is negative and white is positive.

Output: The code, and the gradient-pair image for both transA and simA.

In [4]:
def find_x_gradient(img):
    gradient = cv2.Sobel(img,cv2.CV_64F,1,0,ksize=5)
    r = np.max(gradient) - np.min(gradient)
    gradient = 255 * (gradient - np.min(gradient)) / r
    return gradient

In [5]:
def find_y_gradient(img):
    gradient = cv2.Sobel(img,cv2.CV_64F,0,1,ksize=5)
    r = np.max(gradient) - np.min(gradient)
    gradient = 255 * (gradient - np.min(gradient)) / r
    return gradient

In [6]:
filtered_trans_a = cv2.GaussianBlur(img_trans_a_gray, (7, 7), 1)
trans_a_x = find_x_gradient(filtered_trans_a)
trans_a_y = find_y_gradient(filtered_trans_a)
trans_a_gradient_pair=cv2.hconcat([trans_a_x,trans_a_y])
cv2.imwrite('ps4-1-1-a.png', trans_a_gradient_pair)

True

In [7]:
filtered_sim_a = cv2.GaussianBlur(img_sim_a_gray, (7, 7), 1)
sim_a_x = find_x_gradient(filtered_sim_a)
sim_a_y = find_y_gradient(filtered_sim_a)
sim_a_gradient_pair=cv2.hconcat([sim_a_x,sim_a_y])
cv2.imwrite('ps4-1-1-b.png', sim_a_gradient_pair)

True

![ps4-1-1-a](./ps4-1-1-a.png)

![ps4-1-1-b](./ps4-1-1-b.png)

1.2 Write code to compute the Harris value. You can try the weights just equal to 1. But it might work better with a smoother Gaussian that is higher at the middle and falls off gradually. Your output is a scalar function. Apply to transA, transB, simA, and simB.

Output: The code and the Harris value output image for each of the images. To display the output reasonably you will have to scale the image values to be in a range of 0-255 or 0.0 to 1.0, depending upon how you deal with images.
Finally you can find some corner points. To do this requires two steps: thresholding and non- maximal suppression. You’ll need to choose a threshold value that eliminates points that don’t seem to be plausible corners. And for the non-maximal suppression, you’ll need to choose a radius (could be size of a side of a square window instead of a circle radius) over which a pixel has to be a maximum.

In [8]:
def compute_Harris(img, win_size):
    height, width = img.shape
    R = np.zeros((height - win_size + 1, width - win_size + 1))
    x_gradient = find_x_gradient(img)/255.0
    y_gradient = find_y_gradient(img)/255.0
    alpha = 0.05
    for ix in range(width - win_size + 1):
        for iy in range(height - win_size + 1):
            Ixx = x_gradient[iy:iy+win_size, ix:ix+win_size] ** 2 / (win_size ** 2)
            Ixx = Ixx.sum()
            Iyy = y_gradient[iy:iy+win_size, ix:ix+win_size] ** 2 / (win_size ** 2)
            Iyy = Iyy.sum()
            Ixy = np.dot(x_gradient[iy:iy+win_size, ix:ix+win_size], y_gradient[iy:iy+win_size, ix:ix+win_size]) / (win_size ** 2)
            Ixy = Ixy.sum()
            
            M = [[Ixx, Ixy], [Ixy, Iyy]]
            R[iy, ix] = np.linalg.det(M) - alpha * (np.trace(M) ** 2)
    R_range = np.max(R) - np.min(R)
    R_pic = 255 * (R - np.min(R)) / R_range
    R_pic = abs(128 - R_pic) * 2
    return R_pic

In [9]:
def threshold_Harris(R, t, r):
    R_out = R.copy()
    R_out[np.where(R<t)] = 0

    height, width = R.shape
    for dx in range(width//r):
        for dy in range(height//r):
            ix = dx * r
            iy = dy * r
            w = R_out[iy:iy+r, ix:ix+r]
            local_max = np.max(w)
            arg_max = np.argmax(w)
            y_max = arg_max // r
            x_max = arg_max % r
            R_out[iy:iy+r, ix:ix+r] = 0
            R_out[iy + y_max, ix + x_max] = local_max
    return R_out

In [10]:
R_trans_a = compute_Harris(img_trans_a_gray, HARRIS_WIN_SIZE)
R_trans_a_t = threshold_Harris(R_trans_a, R_THRESHOLD, R_RADIUS)
trans_a_R_pair=cv2.hconcat([R_trans_a,R_trans_a_t])
cv2.imwrite('ps4-1-2-a.png', trans_a_R_pair)

True

In [11]:
R_trans_b = compute_Harris(img_trans_b_gray, HARRIS_WIN_SIZE)
R_trans_b_t = threshold_Harris(R_trans_b, R_THRESHOLD, R_RADIUS)
trans_b_R_pair=cv2.hconcat([R_trans_b,R_trans_b_t])
cv2.imwrite('ps4-1-2-b.png', trans_b_R_pair)

True

In [12]:
R_sim_a = compute_Harris(img_sim_a_gray, HARRIS_WIN_SIZE)
R_sim_a_t = threshold_Harris(R_sim_a, R_THRESHOLD, R_RADIUS)
sim_a_R_pair=cv2.hconcat([R_sim_a,R_sim_a_t])
cv2.imwrite('ps4-1-2-c.png', sim_a_R_pair)

True

In [13]:
R_sim_b = compute_Harris(img_sim_b_gray, HARRIS_WIN_SIZE)
R_sim_b_t = threshold_Harris(R_sim_b, R_THRESHOLD, R_RADIUS)
sim_b_R_pair=cv2.hconcat([R_sim_b,R_sim_b_t])
cv2.imwrite('ps4-1-2-d.png', sim_b_R_pair)

True

![ps4-1-2-a](./ps4-1-2-a.png)

![ps4-1-2-b](./ps4-1-2-b.png)

![ps4-1-2-c](./ps4-1-2-c.png)

![ps4-1-2-d](./ps4-1-2-d.png)

1.3 Write a function to threshold and do non-maximal suppression on the Harris output. Surprise, huh? Adjust the threshold and radius until you get a “nice” set of points, probably on the order of a hundred or two (or three?). But use your judgment in terms of getting enough points. Are there any points that are not found in both images?

Output: The code. Apply your function to both image pair: (transA, transB) and (simA, simB). Mark the corners visibly in each of the four result images and provide those images. Also, describe the behavior of your corner detector including anything surprising, such as points not found in both images of a pair.

In [14]:
def draw_Harris(img, r, t, w):
    img_out = img.copy()
    for ix in range(r.shape[1] - 11):
        for iy in range(r.shape[0] - 11):
            if r[iy, ix] > t and r[iy, ix] != 255:
                img_out = cv2.circle(img_out, (ix+w, iy+w), 5, (0,255,0), 1)
    return img_out

In [15]:
trans_a_harris = draw_Harris(img_trans_a, R_trans_a_t, 128, 5)
cv2.imwrite('ps4-1-3-a.png', trans_a_harris)

True

In [16]:
trans_b_harris = draw_Harris(img_trans_b, R_trans_b_t, 128, 5)
cv2.imwrite('ps4-1-3-b.png', trans_b_harris)

True

In [17]:
sim_a_harris = draw_Harris(img_sim_a, R_sim_a_t, 128, 5)
cv2.imwrite('ps4-1-3-c.png', sim_a_harris)

True

In [18]:
sim_b_harris = draw_Harris(img_sim_b, R_sim_b_t, 128, 5)
cv2.imwrite('ps4-1-3-d.png', sim_b_harris)

True

![ps4-1-3-a](./ps4-1-3-a.png)

![ps4-1-3-b](./ps4-1-3-b.png)

![ps4-1-3-c](./ps4-1-3-c.png)

![ps4-1-3-d](./ps4-1-3-d.png)

Answer:

The R returns an array of negative numbers and 0; after projecting the values from 0 to 255, I found the flat areas are gray and corners are black or white. So I subtracted the matrix with 128 and remapped the values to 0 to 255

2 SIFT features

Now that you have keypoints for both image pairs, we can compute descriptors. You will be glad to know that we do not expect you to write your own SIFT descriptor code. Instead you’ll use either a MATLAB package called VLFeat or the SIFT or SURF classes in OpenCV. See the accompanying supplemental document for details.
The standard use of a SIFT library consists of you just providing an image and the library does its thing: finds interest points at various scales and computes descriptors at each point. We’re going to use the code just to compute the orientation histogram descriptors for the interest points you have already detected from Problem 1. To do so, you need to provide a scale setting and an orientation for each feature point as well as the gradient magnitude and angle for each pixel. The scale we’ll fix to 1.0 (see accompanying SIFT software usage tutorial). The orientation needs to be computed from the gradients:
angle = atan2(Iy,Ix)
But, you already have the gradient images! So you can create an “angle” image and then for a
given feature point at < ui, vi > you can get the gradient direction.


2.1 Write the function to compute the angle. Then for the set of interest points you found above, plot the points for all of transA, transB, simA and simB on the respective images and draw a little line that shows the direction of the gradient. In MATLAB, if you want you can use the VLFeat function vl plotframe to draw the feature points locations and the angle. You’ll need to figure it out - look at http://www.vlfeat.org/overview/sift.html and also the documentation for vl plotframe. In OpenCV you can use the method drawKeypoints().

Output: The code. And both of the drawn on pairs (transA, transB) and (simA, simB).

In [19]:
def compute_orientation(img):
    Ix = find_x_gradient(img)
    Iy = find_y_gradient(img)
    angle = np.arctan2(Iy, Ix) * 360
    return angle

In [20]:
def compute_keypoints(r, angle): 
    keypoints = []
    h, w = r.shape
    r_normalized = r.copy()
    cv2.normalize(r,  r_normalized, 0, 25, cv2.NORM_MINMAX)
    for ix in range(w):
        for iy in range(h):
            if r[iy, ix] > 0:
                a = angle[iy + 5,ix + 5]
                keypoint = cv2.KeyPoint(x=ix+5, y=iy+5 ,_size=r_normalized[iy, ix],_angle = a)
                keypoints.append(keypoint)
    return keypoints

In [21]:
trans_a_angle = compute_orientation(img_trans_a_gray)
trans_a_kp = compute_keypoints(R_trans_a_t, trans_a_angle)

trans_b_angle = compute_orientation(img_trans_b_gray)
trans_b_kp = compute_keypoints(R_trans_b_t, trans_b_angle)

In [22]:
trans_a_kp_img = cv2.drawKeypoints(img_trans_a_gray, trans_a_kp, img_trans_a, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
trans_b_kp_img = cv2.drawKeypoints(img_trans_b_gray, trans_b_kp, img_trans_b, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
trans_kp_img_pair=cv2.hconcat([trans_a_kp_img, trans_b_kp_img])

In [25]:
sift = cv2.SIFT_create()

In [26]:
# Calculate OpenCV SIFT keypoints
trans_a_kp_t = sift.detect(img_trans_a_gray, None)
trans_b_kp_t = sift.detect(img_trans_b_gray, None)

In [27]:
trans_a_kp_img_t = cv2.drawKeypoints(img_trans_a_gray, trans_a_kp_t, img_trans_a.copy(), flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
trans_b_kp_img_t = cv2.drawKeypoints(img_trans_b_gray, trans_b_kp_t, img_trans_b.copy(), flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
trans_kp_img_pair_t = cv2.hconcat([trans_a_kp_img_t, trans_b_kp_img_t])
trans_a_kp_compare = cv2.vconcat([trans_kp_img_pair, trans_kp_img_pair_t])
cv2.imwrite('ps4-2-1-a.png', trans_a_kp_compare)

True

2 images line 1: keypoints by implemented functions

2 images line 2: keypoints by OpenCV SIFT functions
![ps4-2-1-a](./ps4-2-1-a.png)

In [28]:
sim_a_angle = compute_orientation(img_sim_a_gray)
sim_a_kp = compute_keypoints(R_sim_a_t, sim_a_angle)

sim_b_angle = compute_orientation(img_sim_b_gray)
sim_b_kp = compute_keypoints(R_sim_b_t, sim_b_angle)

In [29]:
sim_a_kp_img = cv2.drawKeypoints(img_sim_a_gray, sim_a_kp, img_sim_a, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
sim_b_kp_img = cv2.drawKeypoints(img_sim_b_gray, sim_b_kp, img_sim_b, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
sim_kp_img_pair=cv2.hconcat([sim_a_kp_img, sim_b_kp_img])

In [30]:
# Calculate OpenCV SIFT keypoints & descriptors
sim_a_kp_t = sift.detect(img_sim_a_gray, None)
sim_b_kp_t = sift.detect(img_sim_b_gray, None)

In [31]:
sim_a_kp_img_t = cv2.drawKeypoints(img_sim_a_gray, trans_a_kp_t, img_sim_a, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
sim_b_kp_img_t = cv2.drawKeypoints(img_sim_b_gray, sim_b_kp_t, img_sim_b, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
sim_kp_img_pair_t = cv2.hconcat([sim_a_kp_img_t, sim_b_kp_img_t])
sim_a_kp_compare = cv2.vconcat([sim_kp_img_pair, sim_kp_img_pair_t])
cv2.imwrite('ps4-2-1-b.png', sim_a_kp_compare)

True

2 images line 1: keypoints by implemented functions

2 images line 2: keypoints by OpenCV SIFT functions
![ps4-2-1-b](./ps4-2-1-b.png)

2.2 Write the function to call the appropriate SIFT descriptor extraction function with the necessary input data structures. Do this for all the keypoints in both pairs of images. Then call the matching functions of VLFeat or OpenCV to compute the best matches between the left and right images of each pair. Then create the putative-pair-image for both transA–transB and simA–simB pair. You must write your own drawing function (note you may use OpenCV’s line() function or MATLAB’s plot() function).

Output: The code. And both of the putative-pair-images.

In [32]:
def call_descriptor(img, kp):
    kp, des = sift.compute(img, kp)
    return des

In [33]:
def draw_matches(imgl, kpl, imgr, kpr, matches):
    output = cv2.hconcat([imgl, imgr])
    height, width, color = imgl.shape
    for match in matches:
        xl, yl = kpl[match.queryIdx].pt
        xr, yr = kpr[match.trainIdx].pt
        output = cv2.line(output, (int(xl), int(yl)), (int(xr + width), int(yr)), (0,255,0), 1)
    return output

In [34]:
trans_a_dp = call_descriptor(img_trans_a_gray, trans_a_kp)
trans_b_dp = call_descriptor(img_trans_b_gray, trans_b_kp)

In [35]:
bf = cv2.BFMatcher(crossCheck=True)
trans_matches = bf.match(trans_a_dp, trans_b_dp)
trans_matches = sorted(trans_matches, key = lambda x:x.distance)

In [38]:
img_trans_a = cv2.imread('transA.jpg')
img_trans_b = cv2.imread('transB.jpg')
img_sim_a = cv2.imread('simA.jpg')
img_sim_b = cv2.imread('simB.jpg')

In [39]:
trans_matching_pair = draw_matches(img_trans_a, trans_a_kp, img_trans_b, trans_b_kp, trans_matches[:50])

In [40]:
cv2.imwrite('ps4-2-2-a.png', trans_matching_pair)

True

![ps4-2-2-a](./ps4-2-2-a.png)

In [41]:
sim_a_dp = call_descriptor(img_sim_a_gray, sim_a_kp)
sim_b_dp = call_descriptor(img_sim_b_gray, sim_b_kp)

In [42]:
bf = cv2.BFMatcher(crossCheck=True)
sim_matches = bf.match(sim_a_dp, sim_b_dp)
sim_matches = sorted(sim_matches, key = lambda x:x.distance)

In [43]:
sim_matching_pair = draw_matches(img_trans_a, sim_a_kp, img_trans_b, sim_b_kp, sim_matches[:100])

In [44]:
cv2.imwrite('ps4-2-2-b.png', sim_matching_pair)

True

![ps4-2-2-b](./ps4-2-2-b.png)

3 RANSAC

We’re almost there. You now have keypoints, descriptors and their putative matches. What remains is RANSAC. To do this for the translation case is easy. Using the matched keypoints for transA and transB, randomly select one of the putative matches. This will give you an offset (a translation in X and Y ) between the two images. Find out how many other putative matches agree with this offset (remember, you may have to account for noise, so ”agreeing” means within some tolerance). This is the consensus set for the selected first match. Find the best such translation - the one with the biggest consensus set.

3.1 Write the code to do the translational case on transA and transB. Draw the lines on the adjoined images of the biggest consensus set.

Output: The code and the drawn upon adjoined image pairs. 

Also, say what the translation vector was and what percentages of your matches was the biggest consensus set.

In [104]:
def translation_ransac(imgl, imgr, kpl, kpr, matches):
    outimg = cv2.hconcat([imgl, imgr])
    height, width, color = imgl.shape
    
    n = len(kpl)
    max_matching_count = 0
    max_matching_transx = 0
    max_matching_transy = 0
    max_matching_list = []
    count = 0
    
    for i in range(n):
        rand_matching = np.random.choice(matches)
        rand_xl, rand_yl = kpl[rand_matching.queryIdx].pt
        rand_xr, rand_yr = kpr[rand_matching.trainIdx].pt
        trans_x = rand_xr - rand_xl
        trans_y = rand_xr - rand_yr
        matching_list = []
        for match in matches:
            xl, yl = kpl[match.queryIdx].pt
            xr, yr = kpr[match.trainIdx].pt
            if xl + trans_x - xr < 5 and yl + trans_y - yr < 5:
                matching_list.append(match)
        if len(matching_list) > max_matching_count:
            max_matching_count = len(matching_list)
            max_transx = trans_x
            max_transy = trans_y
            max_matching_list = matching_list
    print("Best matching points:", max_matching_count, "\nKeypoints in left img:", len(kpl),\
          "\nKeypoints in right img:", len(kpr), "\nBest matches:", len(matches))
    for match in max_matching_list:
        xl, yl = kpl[match.queryIdx].pt
        xr, yr = kpr[match.trainIdx].pt
        outimg = cv2.line(outimg, (int(xl), int(yl)), (int(xr + width), int(yr)), (0,255,0), 1)
          
    return outimg

In [105]:
trans_ransac = translation_ransac(img_trans_a, img_trans_b, trans_a_kp, trans_b_kp, trans_matches[:100])

Best matching points: 48 
Keypoints in left img: 503 
Keypoints in right img: 795 
Best matches: 100


In [106]:
cv2.imwrite('ps4-3-1.png', trans_ransac)

True

Answer:

The translation is two vectors in direction x and y, so I didn't use a transition matrix but get the vectors only by subtracting the x and y variables in two images. When finding the corresponding point, I took the point in the left image and added by the translation vector x and y, compare the result with the point in the right image.

I set 5 as the tolerance of the difference, and got 48 best matching points out of 100 pairs of matching point

![ps4-3-1](./ps4-3-1.png)

3.2 Do the same as above but for the similarity pair simA and simB. Write code to apply RANSAC by randomly picking two matches, solving for the transform, and determining the consensus set. Draw the lines on the adjoined images for the biggest consensus set.


Output: The code and the drawn upon adjoined image pairs. 

Also, say what the transform matrix is for best set and what percentages of your matches was the biggest consensus set.

In [125]:
def similarity_ransac(imgl, imgr, kpl, kpr, matches):
    outimg = cv2.hconcat([imgl, imgr])
    height, width, color = imgl.shape
    
    n = len(kpl)
    max_matching_count = 0
    max_matching_matrix = []
    max_matching_list = []
    for i in range(int(0.95*n)):
        rand_matching1, rand_matching2 = np.random.choice(matches, size=2)
        rand_xl1, rand_yl1 = kpl[rand_matching1.queryIdx].pt
        rand_xr1, rand_yr1 = kpr[rand_matching1.trainIdx].pt
        rand_xl2, rand_yl2 = kpl[rand_matching2.queryIdx].pt
        rand_xr2, rand_yr2 = kpr[rand_matching2.trainIdx].pt
        A = np.array([[rand_xl1, rand_yl1, 1, 0], [rand_yl1, -rand_xl1, 0, 1],\
                      [rand_xl2, rand_yl2, 1, 0], [rand_yl2, -rand_xl2, 0, 1]])
        B = np.array([rand_xr1, rand_yr1, rand_xr2, rand_yr2])
        s = np.linalg.pinv(A) * B
        s = s[0]
        matching_list = []
        for match in matches:
            xl, yl = kpl[match.queryIdx].pt
            xr, yr = kpr[match.trainIdx].pt
            tA = np.array([[xl, yl, 1, 0], [yl, -xl, 0, 1]])
            tB = np.array([xr, yr])
            sim = np.matmul(tA, s)
            if np.sum(sim - tB) < 5:
                matching_list.append(match)
        if len(matching_list) > max_matching_count:
            max_matching_count = len(matching_list)
            max_matching_matrix = s
            max_matching_list = matching_list
    print("Best matching points:", max_matching_count, "\nKeypoints in left img:", len(kpl),\
          "\nKeypoints in right img:", len(kpr), "\nBest matches:", len(matches))
    for match in max_matching_list:
        xl, yl = kpl[match.queryIdx].pt
        xr, yr = kpr[match.trainIdx].pt
        outimg = cv2.line(outimg, (int(xl), int(yl)), (int(xr + width), int(yr)), (0,255,0), 1)
          
    return outimg

In [126]:
sim_ransac = similarity_ransac(img_sim_a, img_sim_b, sim_a_kp, sim_b_kp, sim_matches)
cv2.imwrite('ps4-3-2.png', sim_ransac)

Best matching points: 87 
Keypoints in left img: 398 
Keypoints in right img: 263 
Best matches: 87


True

![ps4-3-2](./ps4-3-2.png)

Answer:

For similarity, we only need two pairs of points to make a transition matrix. After calculating, we have:

A = [[x y 1 0]  [y -x 0 1]]

B = [x' y']

A*s = B

By using pinv we obtain s and apply s on all matching points we can have inliers. With a tolerance of 5, 100% percent of matchings are inliers.